## Import libraries and set up functions
Starting directory is this "..\\asset" from this notebook code

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()
import cv2
import matplotlib.pyplot as plt
import random
from keras.backend import clear_session
from keras.backend import get_session
import time
import math
from datetime import timedelta
import mediapipe as mp
import csv
import pandas as pd

os.chdir("..\\asset") # change working directory

In [ ]:
emotionList = ('Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Anger', 'Contempt')
TRAINANNOTATIONPATH = "train_set\\annotations\\"
TRAINIMAGEPATH = "train_set\\images\\"
TRAINLANDMARKPATH = "train_set\\landmarks\\"
TESTANNOTATIONPATH = "val_set\\annotations\\"
TESTIMAGEPATH = "val_set\\images\\"
TESTLANDMARKPATH = "val_set\\landmarks\\"

def LoadAllImageNames(filePath, limit = 0, catLimit = []):
  """Load the images names and label in tuple format (label, image name)

  Args:
      filePath (str): directory of image folder
      limit (int, optional): max number of image to load. Defaults to 0.
      catLimit (list<int>, optional): array of image count limit for each class. Defaults to []*8.

  Returns:
      list<str>: shuffled list of image names
  """
  limitCounter = [0,0,0,0,0,0,0,0]
  dataSet = []
  loadCounter = 0
  for file in os.scandir(filePath):
    if (limit > 0 and loadCounter > limit):
      break
  
    name = file.name[:-4] # file name w/o file extension
    data = np.load("{}{}_exp.npy".format(TRAINANNOTATIONPATH if filePath == TRAINIMAGEPATH else TESTANNOTATIONPATH, name)) # 
    label = int(data.item(0))

    if len(catLimit) > 0 and limitCounter[label] >= catLimit[label]:
        continue
    limitCounter[label] += 1
    loadCounter += 1

    dataSet.append((label, file.name))
    if (loadCounter%10000==0):
      print("Files loaded:{}".format(loadCounter))
  
  print("Total images loaded: ", loadCounter)
  print("Images Loaded: ", limitCounter)
  random.shuffle(dataSet)
  return dataSet

# Load the pixels of a picture to numpy.ndarray format. false for test set, true for training set
# Return image in RGB format
def LoadImage(imagePath, imageName, normalize = True):
  """Load image using numpy

  Args:
      imagePath (str): image path 
      imageName (str): image name
      normalize (bool, optional): To normalize image or not. Defaults to True.

  Returns:
      numpy array: x,y,3 array
  """
  # print("{}{}{}".format(os.getcwd(), "\\"+imagePath, imageName))
  image_array = cv2.imread("{}{}{}".format(os.getcwd(), "\\"+imagePath, imageName))
  image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
  if normalize:
    image_array = image_array/255
    
  return image_array

# Extract the daata from 0 to amount from list and return it
def CropData(list, amount):
  if (len(list) < amount):
    amount = len(list)
  croppedList = list[:amount]
  del list[:amount]
  return croppedList

def LoadImages(imagePath, list):
  label = []
  data = []
  for entries in list:
    try:
      image = LoadImage(imagePath, entries[1])
      data.append(image) 
      label.append(entries[0]) 
    except: 
      print("Failed to load training image: ", entries[1])
  npLabel = np.array(label) 
  npData = np.array(data) 
  return npLabel, npData 

# Load a list of face mesh in landmark_pb2.NormalizedLandmarkList format
def LoadFaceMeshes(faceMesh, path, list, withFace = False):
  face = []
  label = []
  data = []
  for entry in list:
    # try:
    if withFace:
      face.append(LoadImage(path, entry[1], normalize = False)) # Load image
      data.append(faceMesh.process(face[-1])) # process it into face mesh and append into data
    else:
      data.append(faceMesh.process(LoadImage(path, entry[1], normalize = False))) # Load image, process it into face mesh, and append into data
    label.append(entry[0])
    # except:
      # print("Failed to load training image: ", entry[1])

  if withFace:
    return np.array(label), np.array(data), np.array(face)
  else:
    return np.array(label), np.array(data)

# Load face mesh data in numpy array of 
def LoadFaceMeshVertices(faceMesh, path, list):
  label = []
  data = []
  for entry in list:
    # load image, process it into normalizedlandmarklist format, obtain face data from the list, convert it into vertices
    data.append(MultiLandmarkToVertices(faceMesh.process(LoadImage(path, entry[1], normalize = False)).multi_face_landmarks[0]))
    label.append(entry[0])
    # except:
      # print("Failed to load training image: ", entry[1])

  return np.array(label), np.array(data)
  
def MultiLandmarkToVertices(multiLandmark):
  verticesList = []
  for idx, landmark in enumerate(multiLandmark.landmark):
    verticesList.append([landmark.x, landmark.y, landmark.z])

  return verticesList


def InitializeModel():
  pretrained_model = tf.keras.applications.MobileNetV3Large(input_shape=(224,224,3)) # Initializing model with mobile net V3 pretrained model

  # Initializing the input and output from the model, removing last layer
  base_input = pretrained_model.layers[0].input
  base_output = pretrained_model.layers[-2].output

  # Adding 3 more layers to output side
  final_output = layers.Dense(128)(base_output) # Adding new layers, to the output side
  final_output = layers.Activation('relu')(final_output) # activating layer
  final_output = layers.Dense(64)(final_output)
  final_output = layers.Activation('relu')(final_output) # activating layer
  final_output = layers.Dense(8, activation = 'softmax')(final_output) # 8 cuz there are 8 image classifications

  new_model = keras.Model(inputs = base_input, outputs = final_output)
  # new_model.summary()
  return new_model

def ConvertToGray(image):
  image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
  return image

def ScaleImage(image, width):
  ratio = image.shape[1]/width
  image = cv2.resize(image, (width, int(image.shape[0]/ratio)))
  return image

def DetectFace(image):
  face_roi = np.ndarray(1)
  faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
  grayImage = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  faces = faceCascade.detectMultiScale(grayImage, 1.3, 5)
  for x,y,w,h in faces:
    roi_gray = grayImage[y:y+h, x:x+w]
    roi_color = image[y:y+h, x:x+w]
    cv2.rectangle(image, (x,y), (x+w, y+h), (255,0,0), 2)
    facess = faceCascade.detectMultiScale(roi_gray)
    if (len(facess) == 0):
      print("Face not detected")
    else:
      for (ex,ey,ew,eh) in facess:
        face_roi = roi_color[ey:ey+eh, ex:ex+ew]
  return face_roi

def ConvertToInput(image):
  input = ScaleImage(image, 224)
  input = np.expand_dims(input, axis = 0) ## to add fourth dimension to fit model input
  input = input/255
  return input

def GetResult(model, input):
  Predictions = model.predict(input)
  print(Predictions)
  result = np.argmax(Predictions)
  return emotionList[result]

def printDataSetLabels(dataSet):
    counterList = list(range(8))
    for label, name in dataSet:
        counterList[label] += 1
    print(counterList)
    
# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    # try:
    #     del classifier # this is from global space - change this as you need
    # except:
    #     pass

    # use the same config as you used to create the session
    tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.9, visible_device_list="0")

class TimeHistory(keras.callbacks.Callback):
  def on_train_begin(self, logs={}):
    self.times = []

  def on_epoch_begin(self, batch, logs={}):
    self.epoch_time_start = time.time()

  def on_epoch_end(self, batch, logs={}):
    self.times.append(time.time() - self.epoch_time_start)

  def AverageTime(self):
    sum = 0
    for time in self.times:
      sum += time
    return sum/len(self.times)

def DrawFaceLandmark(label, image, landmarks):
  mp_drawing = mp.solutions.drawing_utils
  mp_drawing_styles = mp.solutions.drawing_styles
  mp_face_mesh = mp.solutions.face_mesh
  print(f'Face landmarks of {label}:')
  if not landmarks.multi_face_landmarks:
    print("unagle to locate face landmark for {}".format(label))
  annotated_image = image.copy()
  for face_landmarks in landmarks.multi_face_landmarks:
    mp_drawing.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks,
        connections=mp_face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_tesselation_style())
    mp_drawing.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks,
        connections=mp_face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks,
        connections=mp_face_mesh.FACEMESH_IRISES,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_iris_connections_style())
  plt.imshow(annotated_image)

## Setup directory and import image file names

In [ ]:
# Get currect directory (os.getcwd() -> C:\Users\jazzt\src)

#-----------------------Start of code---------------------------
# initialise image names and label
mainTrainSet = LoadAllImageNames(TRAINIMAGEPATH, catLimit=[100]*8)

## Compiling and training model
Using pretrained mode from MobileNetV3, fine-tuning of model can be done in the first block.
In the second block, training of model can be done while tweaking the training parameters such as batch size and epoch.

In [ ]:
pretrained_model = tf.keras.applications.MobileNetV3Large(input_shape=(224,224,3)) # Initializing model with mobile net V3 pretrained model

# Initializing the input and output from the model, removing last layer
base_input = pretrained_model.layers[0].input
base_output = pretrained_model.layers[-2].output

#-------------------------------------------------
# Customizing layers
#-------------------------------------------------
# Adding 3 more layers to output side
final_output = layers.Dense(128)(base_output) # Adding new layers, to the output side
final_output = layers.Activation('relu')(final_output) # activating layer
final_output = layers.Dense(8, activation = 'softmax')(final_output) # 8 cuz there are 8 image classifications

model = keras.Model(inputs = base_input, outputs = final_output)

In [ ]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])

time_callback = TimeHistory()
dataSet = mainTrainSet.copy()

# Initializing model fit params
batchSize = 4
imgPerIter = batchSize*16

count = 0
while(len(dataSet) != 0):
  croppedList = []
  try:
    print("loading image") # Crop and load images in
    croppedList = CropData(dataSet, imgPerIter)
    label, data = LoadImages(TRAINIMAGEPATH, croppedList)
    
    # Training model
    model.fit(data, label, epochs = 11, batch_size = batchSize, callbacks = [time_callback])
    print(time_callback.AverageTime())
  except RuntimeError as e:
    print(e)
  
  # Saving weights
  model.save_weights('bs48_128Dense.h5')
  
  # Print summary of current iteration
  count = count + len(croppedList)
  timeLeft = len(dataSet)/imgPerIter * time_callback.AverageTime()
  print("trained image count: ", count)
  print("Images Left: ", len(dataSet))
  print("Estimated completion time: ", str(timedelta(seconds=timeLeft)))
  
  reset_keras()


## Testing a random image with a face
The following block of code loads a image, crop the image to just the face, and convert it into model input before inserting it into the model

In [ ]:
# Testing trained model
image = LoadImage(TESTIMAGEPATH,"3.jpg", normalize = False)
# image = cv2.resize(image, (224,224))
# image = ScaleImage(image, 224)

face = DetectFace(image)

plt.imshow(face)

preppedInput = ConvertToInput(face)
result = GetResult(model, preppedInput)


In [ ]:
# testing loading in data from tar using tensorflow
# os.chdir("C:\\Users\\j.teoh\\Desktop\\tflite-facial-expression") # change working directory
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, rotation_range=20)

## Test Model
The following block of code takes in a bunch of weight names and test their accuracy and lose before printing the result out.
Insert the weight file names into "modelNames".

In [ ]:
# Testing model

modelNames = ["Limit30000Label126BatchSize48gpu.h5"]
model = InitializeModel()
mainData = LoadAllImageNames(TESTIMAGEPATH)
resultPool =[]

for modelName in modelNames:
    model.load_weights(modelName)
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])

    # initialise image names and label
    testSetData = []
    testSetData.extend(mainData)
    # print(mainData)
    # test model
    lostSum = 0
    accuracySum = 0
    count = 0
    while(len(testSetData) != 0):
      # training data
      try:
        croppedList = CropData(testSetData, 100)
        print("loading image")
        label, data = LoadImages(TESTIMAGEPATH, croppedList)
        result = model.evaluate(data, label, batch_size = 1)
        lostSum += result[0]
        accuracySum += result[1]
      except:
        print("Failed to train data")

      count += 1
      reset_keras()
      # print(count)
    # print(mainData)
    print("==========FINISH TESTING===========")
    print("model name: ", modelName)
    print("average lost: ", lostSum/count)
    print("average accuracy: ", accuracySum/count)
    resultPool.append((modelName, lostSum/count, accuracySum/count))
    
print(resultPool)

## Setting up image with Keras for MobileNet

In [ ]:
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input

# preparing dataset of 1000 images
dataset = trainingSetData[:1000].copy()
trainingSet = []
for label, fileName in dataset:
    # Load image and convert to 224,224,3 nested array
    tempImg = image.load_img("train_set\\images\\"+fileName, target_size = (224,224))
    tempImg = image.img_to_array(tempImg)
    
    # turn all the values in nested array into value between -1 and 1
    trainingSet.append(preprocess_input(tempImg))
    




## GPU Setting
Uncomment the first line of code to force the kernel to compute with CPU instead of GPU

In [ ]:
# tf.config.set_visible_devices([], 'GPU')

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    # tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
    # config = tf.compat.v1.ConfigProto()
    # config.gpu_options.per_process_gpu_memory_fraction = 0.6
    # keras.set_session(tf.Session(config=config))
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

## Mediapipe Testings

In [ ]:
mp_face_mesh = mp.solutions.face_mesh

In [ ]:
help(mp_face_mesh)

In [ ]:
# this code load a list of face data in facemesh vertices and save it
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, refine_landmarks=True)
faceMeshDataSet = []
faceMeshDataSet.extend(mainTrainSet)

print("croppedList: ", len(croppedList))
# Data containing label, data
label, landmarkVertices = LoadFaceMeshVertices(face_mesh, TRAINIMAGEPATH, faceMeshDataSet)
print("label: ", len(label))
print("landmarkVertices: ", len(landmarkVertices))

count = 0
for name, vertices in zip(croppedList, landmarkVertices):
    count += 1
    np.save("{}{}_landmark".format(TRAINLANDMARKPATH, name[1][:-4]), vertices)
    # loadedData = np.load("inputSample1.npy")
    # print(loadedData)

print(count)

In [ ]:
# Load drawing_utils and drawing_styles
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, refine_landmarks=True)

faceMeshDataSet = []
faceMeshDataSet.extend(mainTrainSet)
croppedList = CropData(faceMeshDataSet, 15)

# Data containing label, face image in rgb and normalizedlandmarklist
label, landmarklists, faces = LoadFaceMeshes(face_mesh, TRAINIMAGEPATH, croppedList, True)
DrawFaceLandmark(label[0], faces[0], landmarklists[0])

# Data containing label, data
label2, landmarkVertices = LoadFaceMeshVertices(face_mesh, TRAINIMAGEPATH, croppedList)

# print(landmarkVertices[0])


# np.savetxt('inputSample.txt', np.array(data[0].multi_face_landmarks), delimiter=',')
# def write_test5(vertices, file_path):
# df = pd.DataFrame(data=np.array(landmarkVertices[0]))
# df.insert(loc=0, column='v', value="v")
# df.to_csv('inputSample.csv', index=False, sep=" ", header=False)

np.save("inputSample1", landmarkVertices[0])
loadedData = np.load("inputSample1.npy")
print(loadedData)
# for label, face, mesh in zip(label, faces, data):
#   DrawFaceLandmark(label,face,mesh)


In [ ]:
landmarks = data[0].multi_face_landmarks[0]
for idx, landmark in enumerate(landmarks.landmark):
    print(type(idx), idx)
    print(type(landmark), landmark)
    print(type(landmark.x), landmark.x)
    print(type(landmark.y), landmark.y)
    print(type(landmark.z), landmark.z)
    
    

In [ ]:

image = LoadImage(TESTIMAGEPATH,"3.jpg", normalize = False)
vertices = MultiLandmarkToVertices(face_mesh.process(LoadImage(TESTIMAGEPATH, "3.jpg", normalize = False)).multi_face_landmarks[0])

In [ ]:
# Run MediaPipe Face Mesh.
results = face_mesh.process(image)
print(type(image))
print(type(results))

In [ ]:
DrawFaceLandmark("3", image, results)

In [ ]:
faceMeshDataSet = []
faceMeshDataSet.extend(mainTrainSet)
croppedList = CropData(, 100)
print("loading image")
label, data = LoadImages(TESTIMAGEPATH, croppedList)